In [ ]:
import cv2
import numpy as np
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import math
import glob
import json
import time
import os

# Define camera parameters

In [ ]:
# Camera intrinsic mattrix
camera_matrix = np.array([[606.90486004,   0.        , 326.68717083],
                          [  0.        , 608.62302034, 249.21760803],
                          [  0.        ,   0.        ,   1.        ]])
# Distorsion parameters
dist_coeffs = np.array([[ 1.11494374e-01,  5.93164264e-02,  3.38038952e-03,-3.76295683e-05, -1.07614091e+00]])

# Function to project object mask

In [ ]:
def ply_vtx(pth):
    f = open(pth)
    assert f.readline().strip() == "ply"
    f.readline()
    f.readline()
    f.readline()
    N = int(f.readline().split()[-1])
    while f.readline().strip() != "end_header":
        continue
    pts = []
    for _ in range(N):
        pts.append(np.float32(f.readline().split()[:3]))
    return np.array(pts)

def get_model_points():
    pointxyz = ply_vtx(r".\powerdrill.ply")
    dellist = [j for j in range(0, len(pointxyz))]
    dellist = random.sample(dellist, len(pointxyz) - 2000)
    pointxyz = np.delete(pointxyz, dellist, axis=0)
    return pointxyz

def project_p3d(p3d, cam_scale, K):
    if type(K) == str:
        K = intrinsic_matrix[K]
    p3d = p3d * cam_scale
    p2d = np.dot(p3d, K.T)
    p2d_3 = p2d[:, 2]
    p2d_3[np.where(p2d_3 < 1e-8)] = 1.0
    p2d[:, 2] = p2d_3
    p2d = np.around((p2d[:, :2] / p2d[:, 2:])).astype(np.int32)
    return p2d

def draw_p2ds(img, p2ds, r=1, color=[(255, 0, 0)]):
    if type(color) == tuple:
        color = [color]
    if len(color) != p2ds.shape[0]:
        color = [color[0] for i in range(p2ds.shape[0])]
    h, w = img.shape[0], img.shape[1]
    for pt_2d, c in zip(p2ds, color):
        pt_2d[0] = np.clip(pt_2d[0], 0, w)
        pt_2d[1] = np.clip(pt_2d[1], 0, h)
        img = cv2.circle(
            img, (pt_2d[0], pt_2d[1]), r, c, -1
        )
    return img

# Project mask on image

In [ ]:
# Load image
images = glob.glob('path_to_images')

# Load annotations, in .json
data = [json.loads(line) for line in open('path_to_annotations', 'r')]

aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL) 
parameters = cv2.aruco.DetectorParameters_create()

# Loop over all images
i = 0
for image_file in images:
    frame = cv2.imread(image_file)
    
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters) 
    
    number = int(os.path.splitext(os.path.split(image_file)[-1])[0])

    pose_object = np.array(data[number]["transformation4x4"])[0] 

    mesh_pts = get_model_points().copy()
    mesh_pts = np.dot(mesh_pts, pose_object[:3, :3].T) + pose_object[:3, 3]

    mesh_p2ds = project_p3d(mesh_pts, 1.0, camera_matrix)

    color = (0,0,255)
    frame = draw_p2ds(frame, mesh_p2ds, color=color)

    cv2.imshow('frame', frame)

    cv2.waitKey(0)
    i = i + 1

cv2.destroyAllWindows()

In [ ]:
# Load image
images = glob.glob('path_to_images')

# Load annotations, in .json
data = [json.loads(line) for line in open('path_to_annotations', 'r')]

aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
parameters = cv2.aruco.DetectorParameters_create()

# Loop over all calibration images
i = 0
rvec, tvec = 0, 0
for image_file in images:

    frame = cv2.imread(image_file)
    
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None:
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)

        for i in range(len(ids)):
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.1)

        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        aruco_board = cv2.aruco.GridBoard.create(6, 9, 0.05, 0.01, aruco_dict, 0) 
        _, rvec, tvec = cv2.aruco.estimatePoseBoard(corners, ids, aruco_board, camera_matrix, dist_coeffs, rvec, tvec)
        
        first_pose = np.eye(4)
        first_pose[:3,:3] = cv2.Rodrigues(rvec)[0] 
        first_pose[:3,3] = tvec[:,0] 

        cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.3)
    
    number = int(os.path.splitext(os.path.split(image_file)[-1])[0])

    pose_object = np.array(data[number]["transformation4x4"])[0] 

    mesh_pts = get_model_points().copy()
    mesh_pts = np.dot(mesh_pts, pose_object[:3, :3].T) + pose_object[:3, 3]

    mesh_p2ds = project_p3d(mesh_pts, 1.0, camera_matrix)

    color = (0,0,255)
    frame = draw_p2ds(frame, mesh_p2ds, color=color)

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.waitKey(0)
    i = i + 1

cv2.destroyAllWindows()